In [2]:
import re

In [7]:
def sequence_compare(seq_a, seq_b):
    len1= len(seq_a)
    len2= len(seq_b)
    matches = 0
    for pos in range (0,min(len1,len2)) :
          if seq_a[pos] == seq_b[pos]:
              matches += 1
    return(float(matches)/min(len1,len2))

In [4]:
samInput = open('/Users/robin/GitHub/SLR-HiC/data/BC001_sorted.sam','r')
prime5 = {} # key = chr, val = [soft clipped, rest]
prime3 = {}
for e,line in enumerate(samInput):
    fields = line.rstrip().split("\t")
    cigar = fields[5]
    seq = ''
    S5 = True # if softclipped is 5': true, if softclipped is 3': false
    GATCfound = False
    seq2 = ''
    if "S" in cigar:
        S = re.split('(\d+)',cigar)
        S = filter(None, S)
        idx = S.index("S")
        if idx == 1:
            se = fields[9][   0  :  int(S[0])+4   ]
            if 'GATC' in se:
                GATCstart = se.index('GATC')
                seq = se[:GATCstart+4]
                GATCfound = True
                seq2 = fields[9][    int(S[0]) :   ]
        elif idx == len(S)-1:
            S5 = False
            se = fields[9][((len(fields[9]))-int(S[len(S)-2]))-4:(len(fields[9]))] 
            if 'GATC' in se:
                GATCstart = se.index('GATC')
                seq = se[GATCstart:]
                GATCfound = True
                seq2 = fields[9][     : ((len(fields[9]))-int(S[len(S)-2]))  ]
        if GATCfound == True and len(seq) > 6 and S5 == True:
            span = "-".join([fields[3],str(int(fields[3])+ len(fields[9]))]) # could be on gatc pos, but doesn't matter with SE65bp
            prime5[":".join([fields[2],str(span)])] = [seq,seq2]
        elif GATCfound == True and len(seq) > 6 and S5 != True:
            span = "-".join([fields[3],str(int(fields[3])+ len(fields[9]))]) # could be on gatc pos, but doesn't matter with SE65bp
            prime3[":".join([fields[2],str(span)])] = [seq,seq2]

In [6]:
import pprint
pprint.pprint(prime3['chr11:62112994-62113059'])

['GATCGGTCTGTAGTGATTT', 'CGATAGAATTGTACTGACATGGTTTCCATAGTTTATGCATTAGAATGATC']


In [8]:
for k3,(s3,r3) in prime3.items():
    for k5,(s5,r5) in prime5.items():
        matchS3 = sequence_compare(s3[4:],r5[4:])
        if matchS3 > .7:
            matchS5 = sequence_compare(s5[:len(s5)-4],r3[:len(r3)-4])
            if matchS5 > .7:
                print k3,prime3[k3]
                print k5,prime5[k5]
                print '==='

chr2:60361144-60361209 ['GATCTAAT', 'TTATTAATGTCTTACGGTTAAACACATCCTAAATTTACCTGAGGAGGCTCAAGAGATGATC']
chr1:9414301-9414366 ['TTATTGATC', 'GATCTAAGGGTTTGCAGCTAGATTGGTGTTTTACATGTCTCCTTTGGTAGCATGCAGAAT']
===
chr2:60361144-60361209 ['GATCTAAT', 'TTATTAATGTCTTACGGTTAAACACATCCTAAATTTACCTGAGGAGGCTCAAGAGATGATC']
chr12:9490565-9490630 ['TTTTGATC', 'GATCTTATACATATAGAGAGATCTTCCTTCCTTTACTGGGACCCTTGGCACTGTCCACTGT']
===
chr8:120688411-120688476 ['GATCCTT', 'TTATAACCAAAAGTTTCTGATATCCCGAAAAACTGTATCTATTGTGTGTATTGATTGGGATC']
chr3:4384221-4384286 ['CTATAGATC', 'GATCCTTAAAAGCAAAGAACCAGACAGGGTCAGGTAGATGGGCAAGTAAAAGGGACAAGA']
===
chr9:35125113-35125178 ['GATCATTA', 'TGTCTTAGTCAATAGCCTATCATTTTTCATCTTTGACCTGGTGCTTTGGTCTCCCTGGATC']
chr1:78053928-78053993 ['TGTAGATC', 'GATCATAATTCCAAAACCTTTCCCACTGCTAAAAGCAACCCAAATTTTATTCATTCTTAGC']
===
chr4:10771775-10771840 ['GATCAGTGC', 'CTGTGTTTCAACTTCCACACCTGTTGGTTTGGCTCTCTCCTCTGGATTTGGAGAAAGATC']
chr17:30776650-30776715 ['CTGGGGATC', 'CATTAGTCCAGGGTGGAAGTCAGCTTATTAGAGGGCT']

KeyboardInterrupt: 